# Ideas and thoughts

## Log

2025-07-10 
I am formally starting the project.
I have been looking the last while a bit at the available transportation data for Denmark.
I think it might be interesting to work with railroad data to begin with.
However, for learnings' sake I would be really interested in using open street map, as that seems to be a very nice thing to be proficient with.

Reviewing lecture 9.